In [162]:
import numpy as np
import pandas as pd
import os

current_dir = os.getcwd()
dataset_dir = os.path.join(current_dir, "AHPModel\\Expert Insight Collection.csv")

expert_insights = pd.read_csv(dataset_dir)

In [163]:
expert_insights.head()

,ExpertID,Digital Channel Launch,Technology Stack,Fundraise Strategy,Confidentiality,Responsiveness,Quality Impact,Staff Expertise,Cultural Linguistic Needs,Age Appropriate Guidance,Accessibility
0,1,4,5,6,5,3,4,5,3,4,4
1,2,4,5,6,4,4,5,4,5,2,6
2,3,3,5,5,5,3,6,3,6,4,6
3,4,3,5,4,6,2,4,1,4,2,6
4,5,2,5,2,6,3,3,2,2,1,5


In [164]:
# Melt the DataFrame to transform it into the desired structure
melted_expert_insights = pd.melt(expert_insights, id_vars='ExpertID', var_name='Criterion', value_name='Score')

In [165]:
melted_expert_insights.head()

,ExpertID,Criterion,Score
0,1,Digital Channel Launch,4
1,2,Digital Channel Launch,4
2,3,Digital Channel Launch,3
3,4,Digital Channel Launch,3
4,5,Digital Channel Launch,2


In [166]:
criteria = melted_expert_insights['Criterion'].unique()

In [167]:
ahp_matrix = np.zeros(shape=(len(criteria),len(criteria)))
ahp_sum = np.zeros(len(criteria))
ahp_final = np.zeros(len(criteria))

In [168]:
pivoted_expert_insights = melted_expert_insights[['ExpertID','Criterion','Score']].pivot(index='ExpertID', columns='Criterion', values='Score')

In [169]:
for expert_id in range(len(pivoted_expert_insights)):
    for criterion_index in range(len(criteria)):
        for prev_criterion_index in range(criterion_index - 1, -1, -1):
            try:
                criterion_score = float(pivoted_expert_insights.iloc[expert_id].iloc[criterion_index])
                prev_criterion_score = float(pivoted_expert_insights.iloc[expert_id].iloc[prev_criterion_index])
                
                if criterion_score > prev_criterion_score:
                    # Increment the preference count of the current criterion over the previous criterion
                    ahp_matrix[prev_criterion_index, criterion_index] += 1.0
                    # Increment the total preference count of the current criterion
                    ahp_sum[criterion_index] += 1.0
                elif criterion_score < prev_criterion_score:
                    # Increment the preference count of the previous criterion over the current criterion
                    ahp_matrix[criterion_index, prev_criterion_index] += 1.0
                    # Increment the total preference count of the previous criterion
                    ahp_sum[prev_criterion_index] += 1.0
                else:
                    # Increment the preference count of the current criterion and previous criterion equally
                    ahp_matrix[criterion_index, prev_criterion_index] += 0.5
                    ahp_matrix[prev_criterion_index, criterion_index] += 0.5
                    # Increment the total preference count of the current criterion and previous criterion equally
                    ahp_sum[criterion_index] += 0.5
                    ahp_sum[prev_criterion_index] += 0.5
            except:
                pass

df_ahp_matrix = pd.DataFrame(ahp_matrix.copy(), index=criteria, columns=criteria)

for criterion_index1 in range(len(criteria)):
    for criterion_index2 in range(len(criteria)):
        ahp_matrix[criterion_index1, criterion_index2] = ahp_matrix[criterion_index1, criterion_index2] / ahp_sum[criterion_index1]

for criterion_index in range(len(criteria)):
    ahp_final[criterion_index] = sum(ahp_matrix[:, criterion_index])
    
ahp_final_normalized = (ahp_final / np.sum(ahp_final))

df_ahp_final = pd.DataFrame(ahp_final_normalized, index=criteria, columns=['Score'])

df_ahp_final.index = df_ahp_final.index.str.replace(' ', '_').str.lower()
weights_dict = df_ahp_final.to_dict()
for key, value in weights_dict.items():
    round_score = round(weights_dict[key],4)
    weights_dict[key] = round_score